<p>
<img src="../imgs/EII-ULPGC-logo.jpeg" width="430px" align="right">

# **NOTEBOOK 14**
---


# **Modelos del lenguaje basados en redes neuronales artificiales**

## **Modelos seq2seq 1**

La arquitectura seq2seq, o secuencia a secuencia, es un tipo de modelo de red neuronal utilizado en el aprendizaje profundo para convertir secuencias de entrada en secuencias de salida. Es especialmente popular para tareas como traducción automática, generación de texto, y reconocimiento de voz. Se forma mediante la unión de dos componentes principales: un **codificador** y un **decodificador**.

<p align="center">
<img src="imgs/seq2seq_.svg" width="80%">
</p>


#### **Codificador (Encoder)**

El codificador toma la secuencia de entrada y la procesa para generar una representación interna, a menudo en forma de un vector de contexto o una serie de estados ocultos. Aquí está el proceso detallado:

1. **Entrada de la secuencia**: El modelo toma una secuencia de entrada (por ejemplo, una frase en inglés si estamos haciendo traducción inglés-español).

2. **Embedding de palabras**: Cada palabra o token de la secuencia se transforma en un vector mediante una capa de embebido (embedding).

3. **Procesamiento recurrente**: Los vectores embebidos se pasan a través de capas recurrentes (como LSTM o GRU) para procesar la secuencia. En cada paso temporal, la red actualiza su estado oculto basándose en el token actual y el estado oculto anterior.

4. **Captura de contexto**: El último estado oculto de la red (o una combinación de todos los estados ocultos, dependiendo del diseño) se toma como la representación condensada de la secuencia completa. Este vector de contexto captura la información esencial de la entrada.

### **Decodificador (Decoder)**

El decodificador utiliza la representación generada por el codificador para producir la secuencia de salida.

1. **Inicialización**: El decodificador se inicializa con el estado oculto final del codificador. En algunos diseños, el vector de contexto también se usa directamente como parte de la entrada en cada paso del decodificador.

2. **Generación de la secuencia de salida**: En cada paso, el decodificador genera un token de la secuencia de salida.

   - **Estado oculto**: Se actualiza en cada paso basándose en la entrada actual y el estado oculto anterior.
   - **Entrada**: Puede ser el token de salida del paso anterior o un token especial para indicar el inicio de la secuencia.
   - **Salida**: Un vector que se transforma en las probabilidades de los diferentes tokens posibles para el siguiente paso de la secuencia. Esto se hace típicamente con una capa softmax.

3. **Terminación**: El proceso continúa hasta que se genera un token de final de secuencia o se alcanza una longitud máxima de secuencia.

### **Implementación**

Vamos a implementar un sencillo modelo seq2seq que realice la suma de dos números decimales de, como máximo, 3 cifras. La entrada será una cadena de texto con el formato "123+456" y la salida será una cadena de texto con el formato "0579".

#### **Dataset**

In [1]:
import random
import string
import torch

allowed_chars = string.digits + '+'

class Generator():
    def __init__(self) -> None:
        pass

    # Método para crear un ejemplo de entrenamiento
    def sample(self):
        s1 = random.randint(0, 999)
        s2 = random.randint(0, 999)
        r = s1 + s2
        s1_string = str(s1).zfill(3)
        s2_string = str(s2).zfill(3)
        output = str(r).zfill(4)
        input = s1_string + "+" + s2_string
        return input, output
    
    # Método para crear un lote de ejemplos de entrenamiento
    def batch(self, n):
        inputs = []
        outputs = []
        for _ in range(n):
            input, output = self.sample()
            inputs.append(input)
            outputs.append(output)
        return inputs, outputs
    
    # Método para codificar una cadena de caracteres en un tensor one-hot
    def string_to_tensor(self, s):
        tensor = torch.zeros(len(s), len(allowed_chars))
        for i, char in enumerate(s):
            tensor[i, allowed_chars.index(char)] = 1
        return tensor

    # Método para decodificar un tensor one-hot en una cadena de caracteres
    def tensor_to_string(self, tensor):
        _, max_idx = tensor.max(1)
        return ''.join([allowed_chars[i] for i in max_idx])
    
    # Método para generar un lote de ejemplos de entrenamiento codificados
    def batch_to_tensor(self, n):
        seq_in = []
        seq_out = []
        inputs, outputs = self.batch(n)
        # print(inputs, outputs)
        for input, output in zip(inputs, outputs):
            seq_in.append(self.string_to_tensor(input))
            seq_out.append(self.string_to_tensor(output))
        return torch.stack(seq_in), torch.stack(seq_out)

#### **Modelo**

In [18]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence


# Definir la arquitectura del modelo seq2seq
class Seq2Seq(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Seq2Seq, self).__init__()
        self.hidden_size = hidden_size
        self.encoder = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.decoder = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.output = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden=None):
        dummy_input = input + torch.empty(self.encoder.input_size)
        dummy_input = pad_sequence(dummy_input, batch_first=True, padding_value=0)
        input = dummy_input[:][:-1]
        _, (hn_enc, cn_enc) = self.encoder(input, hidden)
        latent_tensor = hn_enc[0].unsqueeze(1).repeat(1, 4, 1)
        out_dec, (_, _) = self.decoder(latent_tensor, (hn_enc, cn_enc))
        out = F.softmax(self.output(out_dec), dim=2)
        return out
    
model = Seq2Seq(input_size=len(allowed_chars), hidden_size=128, output_size=len(allowed_chars))

#### **Entrenamiento**

In [19]:
import matplotlib.pyplot as plt

history = []

# Bucle de entrenamiento
def train(model, optimizer, loss_fn, n_epochs, batch_size):
    for epoch in range(n_epochs):
        total_loss = 0
        
        optimizer.zero_grad()
        x, y = dg.batch_to_tensor(batch_size)
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        # Print the loss every 10 epochs
        if epoch % 100 == 0:
            print("Epoch: {}, Loss: {}".format(epoch, total_loss))
            history.append(total_loss)

# Definir la función de pérdida y el optimizador
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Entrenar el modelo
dg = Generator()
train(model, optimizer, loss_fn, 5000, 128)


plt.plot(history, label='loss')

RuntimeError: The size of tensor a (127) must match the size of tensor b (128) at non-singleton dimension 0

#### **Evaluación**

In [4]:
# Evaluar el modelo
def evaluate(model, n):
    x, y = dg.batch_to_tensor(n)
    y_pred = model(x)
    for i in range(n):
        print(dg.tensor_to_string(x[i]), dg.tensor_to_string(y_pred[i]), dg.tensor_to_string(y[i]))

evaluate(model, 10)

286+216 0502 0502
095+139 0234 0234
481+531 1012 1012
702+597 1299 1299
407+836 1243 1243
199+879 1078 1078
260+681 0941 0941
176+298 0474 0474
076+017 0093 0093
611+962 1573 1573


Es importante destacar que el modelo no realiza una suma de números siguiendo un procedimiento aritmético, sino que aprende a generar la secuencia de salida correcta mediante un proceso de traducción aprendido a partir de los datos de entrenamiento.

---

### Ejercicio 1

Modifica el código anterior para que el modelo pueda recibir secuencias de longitud variable. Para ello, usa el generador de datos siguiente. Otra de las cosas que debes tener en cuenta es que el generador devuelve una lista de tensores, esto es así para que puedas usar la función `pad_sequence` para rellenar las secuencias más cortas con ceros. Consulta la referencia de la función para ver cómo se usa. El parámetro `batch_first` es importante para que la función sepa si los datos están en formato `(batch_size, seq_len, input_size)` o `(seq_len, batch_size, input_size)`. Nos interesa que estén en el primer formato.



---

In [5]:
import random
import string
import torch

allowed_chars = string.digits + '+' + '#'  # <-- Añadimos el caracter # como padding para la salida

class Generator2():
    def __init__(self) -> None:
        pass

    # Método para crear un ejemplo de entrenamiento
    def sample(self):
        s1 = random.randint(0, 9999)
        s2 = random.randint(0, 9999)
        r = s1 + s2
        s1_string = str(s1)
        s2_string = str(s2)
        output = str(r)
        input = s1_string + "+" + s2_string
        return input, output
    
    # Método para crear un lote de ejemplos de entrenamiento
    def batch(self, n):
        inputs = []
        outputs = []
        for _ in range(n):
            input, output = self.sample()
            inputs.append(input)
            outputs.append(output)
        return inputs, outputs
    
    # Método para codificar una cadena de caracteres en un tensor one-hot
    def string_to_tensor(self, s):
        tensor = torch.zeros(len(s), len(allowed_chars))
        for i, char in enumerate(s):
            tensor[i, allowed_chars.index(char)] = 1
        return tensor

    # Método para decodificar un tensor one-hot en una cadena de caracteres
    def tensor_to_string(self, tensor):
        _, max_idx = tensor.max(1)
        return ''.join([allowed_chars[i] for i in max_idx])
    
    # Método para generar un lote de ejemplos de entrenamiento codificados
    def batch_to_tensor(self, n):
        seq_in = []
        seq_out = []
        inputs, outputs = self.batch(n)
        for input, output in zip(inputs, outputs):
            seq_in.append(self.string_to_tensor(input))
            seq_out.append(self.string_to_tensor(output))
        return seq_in, seq_out # <-- Devolvemos dos listas de tensores

In [6]:
gen = Generator2()
batch_in, batch_out = gen.batch_to_tensor(2)
print(batch_in)
print("-"*50)
print(batch_out)

[tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]]), tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0.